<a href="https://colab.research.google.com/github/SiPilip/pytorch_course/blob/main/04_pytorch_custom_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. PyTorch Custom Dataset

Kita sudah menggunakan beberapa dataset dengan pytorch before

Tetapi bagaiman jika kita ingin menggunakan data sendiri

Salah satu cara untuk melakukannya: Dataset kustom

## Domain Libraries

Bergantung pada apa yang kamu kerjakan, citra, teks, audio, kamu akan melihat tiap PyTorch domain libraries untuk data yang sudah ada berdasarkan fungsi dan kustom data loader

# 0. Import PyTorch dan setup agnostic device

In [4]:
import torch
from torch import nn

torch.__version__

'2.5.1+cu124'

In [5]:
# Setup perangkat agnostik
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
device

'cpu'

## 1. Ambil Data

Dataset kita adalah subset dari Food101 Dataset

Food101 dimulai dari 101 kelas berbeda dari makanan (750 Training, 250 Testing)

Dataset kita dimulai dari 3 kelas dan hanya 10% dari gambar (75 Training, 25 Testing)

Ketika mencoba pembelajaran mesil, ini adalah hal penting untuk melakukan dari kecil lalu meningkatkannya ketika dibutuhkan.

Intinya adalah seberapa cepat kita dapat bereksperiment

In [7]:
import requests
import zipfile
from pathlib import Path

# Buat jalur untuk folder data
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Jika folder gambar tidak ada, download dan siapkan
if image_path.is_dir():
  print(f"{image_path} direktori sudah ada...")
else:
  print(f"{image_path} belum ada, membuat...")
  image_path.mkdir(parents=True, exist_ok=True)

data/pizza_steak_sushi belum ada, membuat...


In [8]:
# Unduh pizza, steak, dan sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/data/pizza_steak_sushi.zip")
  print("Downloading pizza, steak sushi data")
  f.write(request.content)

# Unzip data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping data")
  zip_ref.extractall(image_path)

Unzipping data


## 2. Menjadi satu dengan data (preparasi data, dan eksplorasi data)

In [9]:
import os
def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"Ada {len(dirnames)} direktori dan {len(filenames)} gambar didalam '{dirpath}'.")

In [10]:
walk_through_dir(image_path)

Ada 2 direktori dan 0 gambar didalam 'data/pizza_steak_sushi'.
Ada 3 direktori dan 0 gambar didalam 'data/pizza_steak_sushi/train'.
Ada 0 direktori dan 72 gambar didalam 'data/pizza_steak_sushi/train/sushi'.
Ada 0 direktori dan 78 gambar didalam 'data/pizza_steak_sushi/train/pizza'.
Ada 0 direktori dan 75 gambar didalam 'data/pizza_steak_sushi/train/steak'.
Ada 3 direktori dan 0 gambar didalam 'data/pizza_steak_sushi/test'.
Ada 0 direktori dan 31 gambar didalam 'data/pizza_steak_sushi/test/sushi'.
Ada 0 direktori dan 25 gambar didalam 'data/pizza_steak_sushi/test/pizza'.
Ada 0 direktori dan 19 gambar didalam 'data/pizza_steak_sushi/test/steak'.


In [10]:
# Setup alur latih dan tes

train_dir = image_path / "train"
test_dir = image_path / "test"